<a href="https://colab.research.google.com/github/andisutrakhusnulkarima/DeepLearning/blob/main/Week3_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Andi Sutra Khusnul Karima
#1103220033
                                  #TUGAS 3
#1. Buat model Deep learning MLP (PyTorch and Tensorflow).
#2. Gunakan Matriks Evaluasi (RMSE,MSE,RSquared)
#3. Beri penjelasan untuk setiap persamaan matematika yang ada.

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# 1. Load dataset
url = "https://raw.githubusercontent.com/farrelrassya/teachingMLDL/main/02.%20Deep%20Learning/Dataset/secondhanddataset.csv"
df = pd.read_csv(url).drop(columns=['v.id'])

# 2. Definisikan fitur dan target
X = df.drop(columns=['current price'])
y = df['current price']

# 3. Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Standardisasi fitur
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 5. Konversi ke tensor PyTorch
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_test_torch = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# 6. Buat DataLoader untuk batch training di PyTorch
batch_size = 32
train_dataset = TensorDataset(X_train_torch, y_train_torch)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 7. Definisikan Model MLP di PyTorch
class MLPModel(nn.Module):
    def __init__(self, input_size):
        super(MLPModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.layers(x)

# 8. Inisialisasi model, loss function, dan optimizer
pytorch_model = MLPModel(X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(pytorch_model.parameters(), lr=0.01)

# 9. Training Model PyTorch
epochs = 500
for epoch in range(epochs):
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = pytorch_model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

# 10. Evaluasi Model PyTorch
pytorch_preds = pytorch_model(X_test_torch).detach().numpy()
mse_pytorch = mean_squared_error(y_test, pytorch_preds)
rmse_pytorch = np.sqrt(mse_pytorch)
r2_pytorch = r2_score(y_test, pytorch_preds)

print("\n=== Evaluasi Model PyTorch ===")
print(f'MSE  : {mse_pytorch:.4f}')
print(f'RMSE : {rmse_pytorch:.4f}')
print(f'R²   : {r2_pytorch:.4f}')

# 11. Definisikan Model MLP di TensorFlow
tf_model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)
])

# 12. Compile dan train model TensorFlow dengan early stopping
tf_model.compile(optimizer='adam', loss='mse')
early_stopping = keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
tf_model.fit(X_train, y_train, epochs=500, batch_size=batch_size, verbose=0, callbacks=[early_stopping])

# 13. Evaluasi Model TensorFlow
tf_preds = tf_model.predict(X_test).reshape(-1)
mse_tf = mean_squared_error(y_test, tf_preds)
rmse_tf = np.sqrt(mse_tf)
r2_tf = r2_score(y_test, tf_preds)

print("\n=== Evaluasi Model TensorFlow ===")
print(f'MSE  : {mse_tf:.4f}')
print(f'RMSE : {rmse_tf:.4f}')
print(f'R²   : {r2_tf:.4f}')

# 14. Penjelasan Matematika Evaluasi Model
print("\n=== Penjelasan Matematika ===")
print("1. Mean Squared Error (MSE):")
print("   MSE = (1/n) * Σ(y_true - y_pred)^2")
print("   Mengukur rata-rata kesalahan kuadrat antara nilai asli dan prediksi.")
print("\n2. Root Mean Squared Error (RMSE):")
print("   RMSE = sqrt(MSE)")
print("   Mengubah MSE ke skala yang sama dengan data asli untuk memudahkan interpretasi.")
print("\n3. R-Squared (R²):")
print("   R² = 1 - (SS_res / SS_tot)")
print("   Dengan:")
print("   SS_res = Σ(y_true - y_pred)^2")
print("   SS_tot = Σ(y_true - mean(y_true))^2")
print("   R² mengukur seberapa baik model menjelaskan variabilitas data.")


Epoch 0, Loss: 115766231040.0000
Epoch 100, Loss: 145766944.0000
Epoch 200, Loss: 40087844.0000
Epoch 300, Loss: 12052962.0000
Epoch 400, Loss: 6661136.0000

=== Evaluasi Model PyTorch ===
MSE  : 11522342.6960
RMSE : 3394.4576
R²   : 0.9993


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 

=== Evaluasi Model TensorFlow ===
MSE  : 300698702.7904
RMSE : 17340.6662
R²   : 0.9821

=== Penjelasan Matematika ===
1. Mean Squared Error (MSE):
   MSE = (1/n) * Σ(y_true - y_pred)^2
   Mengukur rata-rata kesalahan kuadrat antara nilai asli dan prediksi.

2. Root Mean Squared Error (RMSE):
   RMSE = sqrt(MSE)
   Mengubah MSE ke skala yang sama dengan data asli untuk memudahkan interpretasi.

3. R-Squared (R²):
   R² = 1 - (SS_res / SS_tot)
   Dengan:
   SS_res = Σ(y_true - y_pred)^2
   SS_tot = Σ(y_true - mean(y_true))^2
   R² mengukur seberapa baik model menjelaskan variabilitas data.
